# SQL Examples Based on Two Medium Articles (AdventureWorks2022, SQL Server)

## Article 1: *How I Learned SQL Better Than 99% of People*  
## ( https://medium.com/data-science-collective/how-i-learned-sql-better-than-99-of-people-fc16af1c0f96 )
### Problem‑Driven, Debugging‑Focused, Real‑World SQL (4 Examples)

---

## **Example 1 — Identify Top 10 Customers by Total Spend**
**Explanation:**  
This example matches the article’s point: “I stopped memorizing syntax and started solving real problems.”  
We calculate which customers spent the most in total.


In [ ]:
SELECT TOP 10 
    c.CustomerID,
    p.FirstName + ' ' + p.LastName AS FullName,
    SUM(soh.TotalDue) AS TotalSpent
FROM Sales.Customer c
JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
GROUP BY c.CustomerID, p.FirstName, p.LastName
ORDER BY TotalSpent DESC;


**Demonstrates:**  
Problem-solving using grouping and aggregation—finding high-value customers as part of real analysis.

---

## **Example 2 — Monthly Revenue Trend (Using Real Data Instead of “Learning Syntax”)**
**Explanation:**  
The article emphasizes solving real questions like “How does revenue change by month?”  
This query calculates total revenue per month.


In [ ]:
SELECT 
    YEAR(OrderDate) AS OrderYear,
    MONTH(OrderDate) AS OrderMonth,
    SUM(TotalDue) AS MonthlyRevenue
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
ORDER BY OrderYear, OrderMonth;


**Demonstrates:**  
Time‑series revenue analysis using `GROUP BY` on dates.

---

## **Example 3 — Debugging Incorrect Logic (Fixing a Broken Query)**
**Explanation:**  
Inspired by the article’s story: “I once wrote DELETE *without* a WHERE clause.”  
Here’s a broken query **and the corrected version**.

### Incorrect Query (double‑counting revenue)

In [ ]:
SELECT SUM(soh.TotalDue)
FROM Sales.SalesOrderHeader soh
JOIN Sales.SalesOrderDetail sod ON soh.SalesOrderID = sod.SalesOrderID;


### Correct Query

In [ ]:
SELECT SUM(soh.TotalDue) as TotalSumDue
FROM Sales.SalesOrderHeader soh;


**Demonstrates:**  
Debugging the incorrect join multiplies rows and inflates totals.

---

## **Example 4 — Ranking Customers Using Window Functions (“Repetition Builds Skill”)**
**Explanation:**  
The article talks about practicing challenges like “rank customers by spending.”  
Here we rank all customers by total lifetime spend.


In [ ]:
WITH CustomerTotals AS (
    SELECT 
        c.CustomerID,
        SUM(soh.TotalDue) AS TotalSpent
    FROM Sales.Customer c
    JOIN Sales.SalesOrderHeader soh ON c.CustomerID = soh.CustomerID
    GROUP BY c.CustomerID
)
SELECT 
    CustomerID,
    TotalSpent,
    RANK() OVER (ORDER BY TotalSpent DESC) AS SpendingRank
FROM CustomerTotals
ORDER BY SpendingRank;


**Demonstrates:**  
Window functions for ranking — one of the article’s core skill‑building techniques.

---

# Article 2: *Writing SQL Like a Pro — Advanced Techniques*  
## (https://medium.com/learning-sql/writing-sql-like-a-pro-advanced-techniques-showcased-in-a-real-life-scenario-2811a848d34a)
### Advanced Transformations Re‑created in Standard SQL (4 Examples)

---

## Example 5 — Merge Price History + Promotion History 
**Explanation:**  
The article merges two “event timelines.”  
Here we simulate that logic using AdventureWorks tables.


In [ ]:
SELECT 
    p.ProductID,
    p.Name AS ProductName,
    plph.StartDate,
    plph.EndDate,
    plph.ListPrice AS BasePrice,
    so.DiscountPct AS PromotionDiscount
FROM Production.Product p
LEFT JOIN Production.ProductListPriceHistory plph 
    ON p.ProductID = plph.ProductID
LEFT JOIN Sales.SpecialOfferProduct sop 
    ON sop.ProductID = p.ProductID
LEFT JOIN Sales.SpecialOffer so 
    ON so.SpecialOfferID = sop.SpecialOfferID;


**Demonstrates:**  
Merging multiple “history” tables into a unified price timeline.

---

## **Example 6 — Filling Price Gaps Using Window Functions (Replacing IGNORE NULLS)**
**Explanation:**  
The article used `LAST_VALUE(... IGNORE NULLS)` but SQL Server doesn’t support it.  
So we emulate it using `FIRST_VALUE` with reverse ordering.


In [ ]:
WITH PriceEvents AS (
    SELECT 
        p.ProductID,
        plph.StartDate,
        plph.ListPrice,
        FIRST_VALUE(plph.ListPrice) OVER (
            PARTITION BY p.ProductID 
            ORDER BY plph.StartDate DESC
        ) AS FilledPrice
    FROM Production.Product p
    LEFT JOIN Production.ProductListPriceHistory plph 
        ON p.ProductID = plph.ProductID
)
SELECT * 
FROM PriceEvents
ORDER BY ProductID, StartDate;



**Demonstrates:**  
Work‑around for BigQuery features using standard SQL server window logic.

---

## **Example 7 — Find Promotion Changes Over Time ("Event Timeline")**
**Explanation:**  
The article transformed ranges → events.  
Here we detect when a promotion *starts* or *ends*.


In [ ]:
SELECT 
    sop.ProductID,
    sop.SpecialOfferID,
    so.StartDate,
    so.EndDate,
    CASE 
        WHEN LAG(so.StartDate) OVER (
                PARTITION BY sop.ProductID 
                ORDER BY so.StartDate
             ) IS NULL 
        THEN 'Promotion Starts'
        ELSE 'Promotion Continues / Changes'
    END AS EventStatus
FROM Sales.SpecialOfferProduct sop
JOIN Sales.SpecialOffer so 
    ON sop.SpecialOfferID = so.SpecialOfferID
ORDER BY 
    sop.ProductID, 
    so.StartDate;



**Demonstrates:**  
Detecting event boundaries using `LAG`.

---

## **Example 8 — Final Unified Table with Effective Price (Promotion Applied)**
**Explanation:**  
This mimics the article’s final output: “price with promotion included.”


In [ ]:
SELECT 
    p.ProductID,
    p.Name AS ProductName,
    plph.StartDate,
    plph.EndDate,
    plph.ListPrice AS BasePrice,
    so.DiscountPct,
    -- Apply discount when present
    CASE 
        WHEN so.DiscountPct IS NOT NULL 
             THEN plph.ListPrice * (1 - so.DiscountPct)
        ELSE plph.ListPrice
    END AS EffectivePrice
FROM Production.Product p
LEFT JOIN Production.ProductListPriceHistory plph 
    ON p.ProductID = plph.ProductID
LEFT JOIN Sales.SpecialOfferProduct sop 
    ON sop.ProductID = p.ProductID
LEFT JOIN Sales.SpecialOffer so 
    ON so.SpecialOfferID = sop.SpecialOfferID
ORDER BY p.ProductID, plph.StartDate;


**Demonstrates:**  
Combining multiple history sources to compute the final promoted price.